# **SECOM Data Analysis and Model Training**

## **0. Download and extract data**

The following cell downloads and extracts the data in case it is not already present in the local directory.

In [1]:
from get_data import download_and_extract_data

url = "https://archive.ics.uci.edu/static/public/179/secom.zip"
download_and_extract_data(url)

2026-01-15 16:32:47,407 - get_data - INFO - secom data not found. Downloading...
2026-01-15 16:32:47,408 - get_data - INFO - Downloading from https://archive.ics.uci.edu/static/public/179/secom.zip...
2026-01-15 16:33:02,090 - get_data - INFO - Download complete.
2026-01-15 16:33:02,093 - get_data - INFO - Extracting data\secom.zip...
2026-01-15 16:33:02,149 - get_data - INFO - Extraction complete.
2026-01-15 16:33:02,151 - get_data - INFO - Cleaned up zip file.
2026-01-15 16:33:02,151 - get_data - INFO - Data ready to be used.


## **1. EDA**